In [ ]:
# 첫번째 셀 - 사용하지 않음
import os

# 현재 작업 디렉토리(경로) 출력
print("현재 경로:", os.getcwd())
path = os.getcwd() + "/pro_data/results/us_economic_data.csv"

# 경로가 존재하는지 확인
if not os.path.exists(path):
    print(f"경로 '{path}'가 존재하지 않습니다.")
else:
    print(f"경로 '{path}'가 존재합니다.")


현재 경로: /workspace/google_cloud/dong


In [22]:
import pandas as pd
import os
from google import genai
from google.genai import types
from dotenv import load_dotenv
import time

# .env 파일 로드
env_path = os.path.join(os.path.dirname('__file__'),'.env')
load_dotenv(dotenv_path=env_path)

# CSV 파일 읽기
df = pd.read_csv("news_selected.csv")
df = df.sort_values(by="일자").reset_index(drop=True)

# 데이터 확인
print(f"전체 데이터: {len(df)}개")
print(f"일자 수: {df['일자'].nunique()}개")
df.head()


전체 데이터: 4875개
일자 수: 359개


,일자,제목,키워드,본문,URL
0,20241001,"[단독] 게임사 29곳, 구글 애플 ‘수수료 갑질’에 집단 대응","게임사,구글,애플,수수료,갑질,집단,대응,법원,집단,조정,제기,예정,수수료,요구,피...",스마트폰에서 앱을 내려받는 앱 마켓을 장악한 구글 애플의 ‘수수료 갑질’로 피해를 ...,https://www.chosun.com/economy/tech_it/2024/10...
1,20241002,"""와~ 무섭게 담네"" 애플 테슬라 판 서학개미 달려든 곳","애플,테슬라,서학개미,엔비디아,테슬라,애플,종목,서학개미,매수세,배당주,ETF,지수...","[머니투데이 천현정 기자] 엔비디아, 테슬라, 애플 등 주요 대형 종목에 쏠렸던 서...",http://news.moneytoday.co.kr/view/mtview.php?n...
2,20241002,“애플도 이용자 데이터 못 본다” 프라이버시 앞세운 애플의 AI [팩플],"애플,이용자,데이터,프라이버시,애플,AI,팩플,시리즈,지난달,아이폰,출시,애플,iO...",지난달 아이폰16 시리즈를 출시한 애플이 iOS(애플 운영체제) 18 업데이트를 통...,https://www.joongang.co.kr/article/25281663
3,20241002,"3500억원 꿀꺽한 애플, 반환도 안해","3500억,애플,반환,안해,애플,앱스토어,입점,개발사,명목,인앱,결제,수수료,350...",애플이 앱스토어에 입점한 국내 개발사들에서 인앱결제 수수료 명목으로 3500억원을 ...,http://www.mk.co.kr/article/11130153
4,20241002,"""애플, 韓 개발사에 부당 수취한 3500억 아직 반환 안해""","애플,개발사,부당,수취,3500억,반환,개발사,부가세,수수료,책정,시정,애플,반환책...",[이데일리 임유경 기자] 애플이 국내 앱 개발사들에게만 부당하게 수취해 논란이 된 ...,http://www.edaily.co.kr/news/newspath.asp?news...


In [14]:
# Gemini 호출 함수
def call_gemini(input_text: str):
    """Gemini API를 호출하여 응답을 받아오는 함수"""
    client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))
    
    model = "gemini-2.0-flash"
    contents = [
        types.Content(
            role="user",
            parts=[types.Part.from_text(text=input_text)],
        ),
    ]
    
    # 스트림이 아닌 일반 응답으로 받기
    response = client.models.generate_content(
        model=model,
        contents=contents,
    )
    
    return response.text


In [23]:
# 일자별로 뉴스 그룹화 및 Gemini 호출 (배치 처리)
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

results = []

# 배치 처리 설정
BATCH_SIZE = 5  # 한번에 처리할 요청 수 (필요시 조정)
BATCH_DELAY = 3  # 배치 간 대기 시간 (초)

# 일자별로 그룹화
grouped = df.groupby('일자')
print("정렬 결과(상위 5개):")
print(df[['일자', '제목']].head())


date_groups = list(grouped)

print(f"처리할 일자 수: {len(date_groups)}개")
print(f"배치 크기: {BATCH_SIZE}개, 배치 간 대기: {BATCH_DELAY}초\n")

# 일자별 데이터 준비
tasks = []
for date, group in date_groups:
    # 해당 일자의 뉴스들을 하나의 텍스트로 합치기
    news_list = []
    for idx, row in enumerate(group.itertuples(), 1):
        news_text = f"뉴스{idx}:\n제목: {row.제목}\n본문: {row.본문}\n"
        news_list.append(news_text)
    
    combined_news = "\n".join(news_list)
    tasks.append((date, combined_news))

# 배치 단위로 처리
total_batches = (len(tasks) + BATCH_SIZE - 1) // BATCH_SIZE
success_count = 0
error_count = 0

# 전체 진행 상황 표시
with tqdm(total=len(tasks), desc="전체 진행", unit="일자") as pbar:
    for batch_idx in range(0, len(tasks), BATCH_SIZE):
        batch = tasks[batch_idx:batch_idx + BATCH_SIZE]
        current_batch = batch_idx // BATCH_SIZE + 1
        
        pbar.set_postfix({"배치": f"{current_batch}/{total_batches}", "성공": success_count, "오류": error_count})
        
        # ThreadPoolExecutor를 사용하여 배치 내에서 병렬 처리
        with ThreadPoolExecutor(max_workers=BATCH_SIZE) as executor:
            # 각 태스크를 제출
            future_to_task = {}
            for date, combined_news in batch:
                future = executor.submit(call_gemini, combined_news)
                future_to_task[future] = (date, combined_news)
            
            # 결과 수집
            for future in as_completed(future_to_task):
                date, combined_news = future_to_task[future]
                try:
                    gemini_response = future.result()
                    results.append({
                        '일자': date,
                        '원본내용': combined_news,
                        '답변': gemini_response
                    })
                    success_count += 1
                    pbar.set_postfix({"배치": f"{current_batch}/{total_batches}", "성공": success_count, "오류": error_count})
                except Exception as e:
                    error_count += 1
                    results.append({
                        '일자': date,
                        '원본내용': combined_news,
                        '답변': f"오류: {str(e)}"
                    })
                    pbar.set_postfix({"배치": f"{current_batch}/{total_batches}", "성공": success_count, "오류": error_count})
                
                pbar.update(1)
        
        # 다음 배치 전에 대기 (마지막 배치가 아닌 경우)
        if batch_idx + BATCH_SIZE < len(tasks):
            time.sleep(BATCH_DELAY)

print(f"\n✅ 처리 완료!")
print(f"   성공: {success_count}개")
print(f"   오류: {error_count}개")
print(f"   총: {len(results)}개")


정렬 결과(상위 5개):
         일자                                        제목
0  20241001       [단독] 게임사 29곳, 구글 애플 ‘수수료 갑질’에 집단 대응
1  20241002           "와~ 무섭게 담네" 애플 테슬라 판 서학개미 달려든 곳
2  20241002  “애플도 이용자 데이터 못 본다” 프라이버시 앞세운 애플의 AI [팩플]
3  20241002                     3500억원 꿀꺽한 애플, 반환도 안해
4  20241002        "애플, 韓 개발사에 부당 수취한 3500억 아직 반환 안해"
처리할 일자 수: 359개
배치 크기: 5개, 배치 간 대기: 3초



전체 진행:   1%|▏         | 5/359 [00:13<16:00,  2.71s/일자, 배치=2/72, 성공=5, 오류=0]


KeyboardInterrupt: 

In [20]:
# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(results)

# 결과 확인
print(f"결과 데이터: {len(result_df)}개")
result_df.head()


결과 데이터: 5개


,일자,원본내용,답변
0,20241005,"뉴스1:\n제목: ""아이폰 16 투명 케이스, 온통 핑크로 물들었는데""..애플 ""이...",## 뉴스 요약:\n\n**뉴스 1: 아이폰 16 핑크 색상 '색빠짐' 논란**\n...
1,20241004,"뉴스1:\n제목: “한국이 만만했나” 해외선 잘내는 구글 애플, 국내 과징금 400...","분석 결과, 다음 주제들이 주요하게 다뤄지고 있습니다:\n\n* **구글과 애플..."
2,20241001,"뉴스1:\n제목: [단독] 게임사 29곳, 구글 애플 ‘수수료 갑질’에 집단 대응\...","## 뉴스1 기사 요약: 국내 게임사 29곳, 구글-애플의 앱 마켓 수수료 갑질에 ..."
3,20241002,"뉴스1:\n제목: 3500억원 꿀꺽한 애플, 반환도 안해\n본문: 애플이 앱스토어에...","주어진 뉴스들을 분석한 결과, 다음과 같이 요약할 수 있습니다.\n\n**주요 토픽..."
4,20241003,"뉴스1:\n제목: “애플, 내년 초 중저가폰 ‘아이폰SE 4세대’ 출시”\n본문: ...",## 뉴스 분석 및 요약\n\n**주요 토픽:**\n\n* **아이폰 SE 4세...


In [ ]:
# 결과를 CSV 파일로 저장
output_filename = "news_gemini_results.csv"
result_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"결과가 '{output_filename}' 파일로 저장되었습니다!")
